---
># AFP_2022_2_K_10_Akropolisz Gépi tanulás framework
---
Hadobás Dávid (TB3376), Kardos Zsolt (O48WRX), Riczkó Henrik (D5GPJ6), Balogh Mihály Viktor (GUFVXA)
---

In [7]:
import math, numpy, random
adatbazis_helye = "xd" #@param {type:"string"}
#@markdown --- 
algoritmus = 'C' #@param ["A", "B", "C"]
aktivaciosfgv = 'sigmoid' #@param ['tanh', 'sigmoid']
print('Választott algoritmus: ', algoritmus)
print('Választott aktivációs függvény: ', aktivaciosfgv)

Választott algoritmus:  C
Választott aktivációs függvény:  sigmoid


## Jelen állapotban az első kódblokkot a drop-down menük opciói kiválasztása után újra kell futtatni!!!!

In [8]:
if aktivaciosfgv == 'tanh':
  def activation(x):
    return numpy.tanh(x)
  def dactivation(x):
    return 1.0 - x**2
elif aktivaciosfgv == 'sigmoid':
  def activation(x):
    return 1.0/(1.0 + numpy.exp(-x))
  def dactivation(x):
    return x*(1.0-x)

'''Bővíthetőség miatt maradnak elif-ek'''
if algoritmus == 'A':
  acti, dacti = activation, dactivation

  ''' Zsolti: Ide jöhet majd az adatbázis olvasás '''
  #samples = [[[0, 0], [0, 0]], [[0, 1], [0, 1]], [[1, 0], [0, 1]], [[1, 1], [1, 1]]]   # AND / OR
  samples = [ [[0, 0], [0, 0, 0]], [[0, 1], [0, 1, 1]], [[1, 0], [0, 1, 1]], [[1, 1], [1, 1, 0]] ]  # AND / OR / XOR

  

  B = 1
  nn = [len(samples[0][0])+B, 4, 3, len(samples[0][1])]
  wl = [ numpy.random.random((nn[l+1], nn[l]))*0.8-0.4 for l in range(len(nn)-1)] 
  delta = [numpy.zeros((nn[l+1])) for l in range(len(nn)-1)]

  epoch = 0
  sumerr = 1.0
  while sumerr>=0.01 and epoch<=10000:
    sumerr = 0.0
    epoch += 1
    for inp, out in samples:
        nl = [ numpy.array(inp + [1.0]*B) ]
        for l in range(len(nn)-1):
            nl.append(acti(numpy.dot(wl[l],nl[l])))
        error = out - nl[-1]
        for l in reversed(range(len(nn)-1)):
            if l == len(nn)-2:
                delta[l][:] = error*dacti(nl[-1])
            else:
                numpy.dot(delta[l+1],wl[l+1], out=delta[l])
                delta[l] *= dacti(nl[l+1])
            wl[l] += 0.5 * delta[l].reshape((nn[l+1],1))*nl[l].reshape((1,nn[l]))
        sumerr += sum(error**2)
  print (epoch,sumerr)
elif algoritmus == 'B':

  acti, dacti = activation, dactivation
  #Adatbázist ide - Henrik
  #samples = [[[0, 0], [0, 0]], [[0, 1], [0, 1]], [[1, 0], [0, 1]], [[1, 1], [1, 1]]]   # AND és OR
  samples = [ [[0, 0], [0, 0, 0]], [[0, 1], [0, 1, 1]], [[1, 0], [0, 1, 1]], [[1, 1], [1, 1, 0]] ]  # AND, OR és XOR

  B = 1
  nn = [len(samples[0][0])+B, 4, 3, len(samples[0][1])]
  wl=[ [ [random.random()*0.8-0.4 for _ in range(nn[l])] for _ in range(nn[l+1])] for l in range(len(nn)-1)] 
  epoch = 0
  sumerr = 1.0
  while sumerr>=0.01 and epoch<=10000:
      sumerr = 0.0
      epoch += 1
      for inp, out in samples:
          nl = [ inp + [1.0]*B ]
          for l in range(len(nn)-1):
              nl.append([acti(sum([nl[l][i] * wl[l][j][i] for i in range(nn[l])])) for j in range(nn[l+1])])
              
          error = [out[j] - nl[-1][j] for j in range(nn[-1])]
          delta = [None for _ in range(len(nn)-1)]
          for l in reversed(range(len(nn)-1)):
              if l == len(nn)-2:
                  delta[l] = [error[j] * dacti(nl[-1][j]) for j in range(nn[-1])]
              else:
                  delta[l] = [sum([delta[l+1][j] * wl[l+1][j][i] for j in range(nn[l+2])])*dacti(nl[l+1][i]) for i in range(nn[l+1])]
              
              for i in range(nn[l]):
                  for j in range(nn[l+1]):
                      wl[l][j][i] += 0.5 * delta[l][j] * nl[l][i]

          sumerr += sum( [error[j]**2 for j in range(nn[-1])])
  print (epoch,sumerr)

elif algoritmus == 'C':
  acti, dacti = activation, dactivation

  w1 = [random.random()*0.4-0.2 for _ in range(3)]
  #Adatbázist ide - Henrik
  #samples = [[[0.0, 0.0], 0.0], [[0.0, 1.0], 1.0], [[1.0, 0.0], 1.0], [[1.0, 1.0], 1.0]] OR
  samples = [[[0.0, 0.0], 0.0], [[0.0, 1.0], 0.0], [[1.0, 0.0], 0.0], [[1.0, 1.0], 1.0]] #AND
  #samples = [[[0.0, 0.0], 0.0], [[0.0, 1.0], 1.0], [[1.0, 0.0], 1.0], [[1.0, 1.0], 0.0]] XOR

  for _ in range(100):
      sumerr = 0.0
      for inp, out in samples:
          x = inp + [1.0]
          y = acti(sum([x[i] * w1[i] for i in range(3)]))
          error = (out - y)
          delta = error * dacti(y)
          for i in range(3):
              w1[i] += 0.8 * delta * x[i]
          sumerr += (y-out)**2
      print (sumerr)

1.091120410005597
1.010294233612718
0.9477963530579934
0.8979572844060801
0.8567450369232786
0.8214452795871171
0.7902792335100293
0.7620985886726745
0.7361695439675714
0.7120259700969791
0.689370907830866
0.6680112586222571
0.6478154618852004
0.6286871126021205
0.6105493850080451
0.5933364327379189
0.5769889890315836
0.5614522878594332
0.546675146041744
0.532609562298859
0.5192105155404698
0.5064358259324958
0.494246029796208
0.4826042552179922
0.4714760968372672
0.46082949057682704
0.4506345888466128
0.44086363638641357
0.43149084691109485
0.4224922809598191
0.4138457256226994
0.40553057699799394
0.39752772628179367
0.3898194503259922
0.38238930735689225
0.37522203836524415
0.3683034744899988
0.3616204505433142
0.3551607246748353
0.34891290405335107
0.3428663763534997
0.33701124677143607
0.3313382802522722
0.3258388485893447
0.3205048820468386
0.3153288251593968
0.3103035963720435
0.305422551198614
0.3006794485950137
0.29606842026357094
0.29158394262544973
0.2872208112187745
0.282974